In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 텐서 플로의 정보 출력 억제하기
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU 장치 지정

# tf.debugging.set_log_device_placement(True)   # 이거 쓰지 마셈 ㅈㄴ 출력 더러움

if not tf.config.list_physical_devices('GPU'):
    print("감지된 GPU가 없습니다. GPU가 없으면 LSTM과 CNN이 매우 느릴 수 있습니다.")

## 10.
_연습문제: 이미지 데이터셋을 하나 선택해 변이형 오토인코더를 훈련하고 이미지를 생성해보세요. 또는 관심있는 레이블이 없는 데이터셋을 찾아서 새로운 샘플을 생성할 수 있는지 확인해 보세요._

### Load MNIST Dataset

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

X_train = X_train.reshape(60000, 28 * 28)
X_test = X_test.reshape(10000, 28 * 28)

X_train.shape, X_test.shape

## Keras Modeling with Fuctional API

#### 'Latent Space' Point Mapping

- 각 이미지가 '잠재공간(Latent Space) 포인트' 주변의 '다변수 정규 분포(Mutilvariate Nodrmal Distribution)'에 매핑

In [ ]:
from keras.layers import Input, Dense
input_img = Input(shape = (784,))
encoded = Dense(256, activation = 'elu')(input_img)
encoded = Dense(128, activation = 'elu')(encoded)

- Variational(Latent Space) Layers
    - 평균(mean)과 분산(log_var)으로 인코딩된 잠재공간(Latent Space) 포인트 분포

In [ ]:
mean = Dense(2, name = 'mean')(encoded)
log_var = Dense(2, name = 'var')(encoded)

#### 'Latent Space' Sampling

- 잠재공간(Latent Space)의 잠재공간-포인트(z) 샘플링 
    - 정규분포상에서 무작위로 선택한 'epsilon'값 사용
        - Encoding 결과값을 그대로 사용하면 항상 같은 결과만 생성
        - 따라서 랜덤 샘플링을 통하여 기존 Data에 존재하지 않는 새로운 Image 생성
- Lambda( ) : 임의의 파이썬 함수 객체를 Keras Layer로 생성 
- K.exp(log_var) : 로그분산 -> 표준편차 변환

In [ ]:
from keras.layers import Lambda

K = keras.backend

def sampling(args):
    mean, log_var = args
    epsilon = K.random_normal(shape = (100, 2), mean = 0., stddev = 1.0)
    
    return mean + K.exp(log_var) * epsilon

z = Lambda(sampling, output_shape = (2,))([mean, log_var]) # Lambda를 사용하면 output이 2개로 나가는 Layer를 만들 수 있다.

#### 'encoder' Model

In [ ]:
encoder = keras.models.Model(input_img, mean) # 모델을 만들 때는 mean 만 출력으로 사용(평균만 뽑아내면 됨)

In [ ]:
encoder.summary()

#### 'generator' Model

- Decoding Layer Structure

In [ ]:
# 오토 인코더 모양으로 똑같이 보여주기 위해 3개 층으로 만들었으나, 인코더랑 층의 개수가 동일하지 않아도 됨
decoder_1 = Dense(128, activation = 'elu')
decoder_2 = Dense(256, activation = 'elu')
decoder_3 = Dense(784, activation = 'sigmoid')

- 랜덤 샘플링 '잠재공간-포인트(Z)' 재구성

In [ ]:
z_sample = decoder_1(z)
z_sample = decoder_2(z_sample)
z_sample = decoder_3(z_sample)

z_sample.shape

- Generator Layers

In [ ]:
decoder_input = Input(shape = (2,))
y_gen = decoder_1(decoder_input)
y_gen = decoder_2(y_gen)
y_gen = decoder_3(y_gen)

- Build 'generator'

In [ ]:
generator = keras.models.Model(decoder_input, y_gen)

In [ ]:
generator.summary()

## VAE Fit

#### 'vae' Model Dense

- Build 'vae' Model
    - End-to-End AutoEncoder

In [ ]:
vae = keras.models.Model(input_img, z_sample)

In [ ]:
vae.summary()

### Model Compile



- Define 'vae_loss'
    - reconstruction_loss : 입력값 재구성 손실
        - **Generator의 Loss**
        - 원본 이미지와 생성된 이미지와의 오차(CEE) 
        - '샘플링 함수'로 생성한 'z' 값으로 얼마나 원본이미지와 유사한 이미지를 잘 생성 하는가?
    - kl_loss : 사전 분포와 잠재 분포 사이의 Kullback Leibler-Divergence(두 확률분포 간 거리)
        - **Encoder의 Loss**
        - 사전 분포(Prior Distribution) : 원본 이미지 확률분포
        - 잠재 분포(Latent Distribution) : 잠재공간 확률분포 
        - '샘플링 함수'의 값(z)이 원본 이미지의 확률분포와 유사한가?

- 추가 설명

In [ ]:
from keras import objectives

reconstruction_loss = objectives.binary_crossentropy(input_img, z_sample)
kl_loss = 0.0005 * K.mean(K.square(mean) + K.exp(log_var) - log_var - 1, axis = -1)
vae_loss = reconstruction_loss + kl_loss

- Add vae_loss

In [ ]:
vae.add_loss(vae_loss)

- Compile with vae_loss

In [ ]:
vae.compile(optimizer = 'adam')

#### Model Training

In [ ]:
%%time
vae.fit(X_train,
       shuffle = True,
       epochs = 300,
       batch_size = 100,
       validation_data = (X_test, None))

## 'Latent Space' Visualization

### Classes in the Latent Space

In [ ]:
X_test_latent = encoder.predict(X_test, batch_size = 100)
plt.figure(figsize = (12, 10))
plt.scatter(X_test_latent[:, 0], X_test_latent[:, 1], c = y_test)
plt.colorbar()
plt.show()

### Display 2D Manifold(20 * 20)
- 두 개의 '개념 벡터(Concept Vector)'로 데이터의 특징을 '표현(Representation)' 
    - 두께, 회전각도 등

In [ ]:
from scipy.stats import norm

n = 20
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))

grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = generator.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize = (10, 10))
plt.imshow(figure, cmap = 'Greys_r')
plt.show()

## 'generator' Test

In [ ]:
encoded_latent = encoder.predict(X_test)  # 'encoder' Test(784 -> 2)
encoded_latent.shape

In [ ]:
generated_imgs = generator.predict(encoded_latent)  # 'generator' Test(2 -> 784)
generated_imgs.shape

### Generating Visualization
- 복원이 아닌 '생성된' 이미지들

In [ ]:
n = 10
plt.figure(figsize = (20, 4))

for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(generated_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()